In [1]:
import os
from os import path

import h5py
import numpy as np
from tqdm import tqdm

from src.data.loaders.ascad import ASCADData
from src.data.preprocess.hw import full_states, hamming_weights
from src.tools.cache import cache_np
from src.trace_set.database import Database
from src.trace_set.set_hw import TraceSetHW

In [2]:
with ASCADData.random_key() as in_file:
    PROF_PLAIN = in_file['Profiling_traces/metadata']['plaintext']
    PROF_KEY = in_file['Profiling_traces/metadata']['key']

    ATT_PLAIN = in_file['Attack_traces/metadata']['plaintext']
    ATT_KEY = in_file['Attack_traces/metadata']['key']

In [3]:
STATES_PROF = cache_np("full_states_ascad_profile", full_states, PROF_PLAIN, PROF_KEY)
STATES_ATT = cache_np("full_states_ascad_attack", full_states, ATT_PLAIN, ATT_KEY)

In [4]:
STATES_PROF.shape

(200000, 10, 16)

In [5]:
TARGET_ROUND = 0
TARGET_BYTE = 2

STATE_PROF = STATES_PROF[:, TARGET_ROUND, TARGET_BYTE]
STATE_ATT = STATES_ATT[:, TARGET_ROUND, TARGET_BYTE]

PROF_PLAIN_BYTE = PROF_PLAIN[:, TARGET_BYTE]
PROF_KEY_BYTE = PROF_KEY[:, TARGET_BYTE]
ATT_PLAIN_BYTE = ATT_PLAIN[:, TARGET_BYTE]
ATT_KEY_BYTE = ATT_KEY[:, TARGET_BYTE]

In [6]:
HW_PROF = hamming_weights(STATE_PROF)
HW_ATT = hamming_weights(STATE_ATT)

In [7]:
# Poor man's speed test
for _ in tqdm(range(10)):
    hamming_weights(STATE_PROF), hamming_weights(STATE_ATT)

100%|██████████| 10/10 [00:00<00:00, 460.59it/s]


Conversion speed is good enough, hamming weight labels are no longer part of the trace files.

### Find files, relabel with plaintext and key.

In [9]:
DB = Database.ascad_none

PLAIN_FIELD = "plaintext_byte"
KEY_FIELD = "key_byte"
STATE_FIELD = "state_byte"

def convert_files(db: Database):
    db_dir = path.dirname(TraceSetHW(db).path)
    for file_name in os.listdir(db_dir):
        file_path = path.join(db_dir, file_name)
        with h5py.File(file_path, 'r') as file:
            if not PLAIN_FIELD in file['profile']:
                assert STATE_FIELD in file['profile'] and STATE_FIELD in file['attack']
                # assert not PLAIN_FIELD in file['profile'] and not PLAIN_FIELD in file['attack']
                # assert not KEY_FIELD in file['profile'] and not KEY_FIELD in file['attack']

                correct_states = np.all(np.array(file['attack'][STATE_FIELD]) == STATE_ATT)
                print(file_name, correct_states)
                assert correct_states

        with h5py.File(file_path, 'r+') as file:
            if not PLAIN_FIELD in file['profile']:
                grp_prof = file['profile']
                grp_att = file['attack']

                if PLAIN_FIELD in grp_prof:
                    del grp_prof[PLAIN_FIELD]
                if KEY_FIELD in grp_prof:
                    del grp_prof[KEY_FIELD]
                if PLAIN_FIELD in grp_att:
                    del grp_att[PLAIN_FIELD]
                if KEY_FIELD in grp_att:
                    del grp_att[KEY_FIELD]

                grp_prof.create_dataset(PLAIN_FIELD, data=PROF_PLAIN_BYTE)
                grp_prof.create_dataset(KEY_FIELD, data=PROF_KEY_BYTE)
                grp_att.create_dataset(PLAIN_FIELD, data=ATT_PLAIN_BYTE)
                grp_att.create_dataset(KEY_FIELD, data=ATT_KEY_BYTE)

        with h5py.File(file_path, 'r') as file:
            assert PLAIN_FIELD in file['profile'] and PLAIN_FIELD in file['attack']
            assert KEY_FIELD in file['profile'] and KEY_FIELD in file['attack']

            correct_plain = np.all(np.array(file['attack'][PLAIN_FIELD]) == ATT_PLAIN_BYTE)
            print(file_name, correct_plain)
            assert correct_plain


convert_files(DB)

default.h5 False


AssertionError: 